In [1]:
## TODO: 
## 1. Get mean and std of dataset - done
## 2. Write a script to plot loss + accuracy graph
## 3. Get FLOPs - done
## 4. Get num layers - done
## ----------------------------------------------------
## 1. Implement differentiable F1 loss function
## 2. Add class weights
## 3. Implement transfer learning part - done
## 4. Implement T-SNE
## 5. Implement gradcam
## 6. Ablation study
## 

In [2]:
import sys
sys.path.append('../../src/')

In [3]:
%load_ext autoreload
%autoreload 2

from train import train_model
from model import initialize_model
from utils import set_requires_grad, save_model
from data import load_data
from plotting import plot_data_loader
from eval import eval_model

In [4]:
import os
import copy
import time
import random
import pickle

import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from tqdm import tqdm

from sklearn.metrics import f1_score, confusion_matrix
from numpy.random import shuffle
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision.io import read_image
import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import torch.autograd.profiler as tprofiler
import torch.utils.data as td

plt.rcParams["savefig.bbox"] = 'tight'

In [5]:
seed = 42

random.seed(seed)

# pytorch RNGs
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
np.random.seed(seed)

In [6]:
data_dir = "../../data"
images_dir = "../../data/covid_pneumonia"

In [7]:
# Models options: resnet50, resnet34, inceptionv3, vgg16, mobile_net_v3_large, efficient_net_b1, efficient_net_b0.
model_name = "efficient_net_b1"

# Number of classes.
num_classes = 3

# Batch Size.
batch_size = 32

# Epochs to train for.
num_epochs = 100

# Number of workers for data loader.
num_workers = 12

# Imagenet norm array passed as default value.
norm_arr=([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# Chest x-ray8 training dataset metrics 
norm_arr=([0.4951, 0.4951, 0.4951], [0.2896, 0.2896, 0.2896])
# Covid Pneumonia dataset metrics
norm_arr = ([0.5159, 0.5159, 0.5159], [0.2554, 0.2554, 0.2554])

# Feature extract flag: False - Tune the whole model,
#                       True - Update only the reshaped layer parameters.
feature_extract = False

# Use pretrained flag: None - Use random weights
#                      String - Use pretrained weights given by String
use_pretrained = models.EfficientNet_B1_Weights.IMAGENET1K_V2

# Initialize the model for this run.
model_pyt, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=use_pretrained)

# lr start and end points for training.
lr_start = 0.01
lr_end = 0.001

# How many epochs to restart.
iter_restart = 10

# Multiplication factor after restart.
mul_restart = 1

# Print the model we just instantiated
#print(model_ft)

In [9]:
data_loaders = load_data(images_dir,
                         batch_size = batch_size, 
                         input_size = input_size, 
                         norm_arr = norm_arr,
                         num_workers = num_workers)

plot_data_loader(data_loader['train'], (2,2))

plot_data_loader(data_loader['test'], (2,2))

plot_data_loader(data_loader['val'], (2,2))

In [10]:
# Check if GPU is available.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Send model to GPU
model_pyt = model_pyt.to(device)

# Find parameters to be updated in this run.
# parameters with requires_grad = True.
params_to_update = model_pyt.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_pyt.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_pyt.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

cuda:0
Params to learn:
	 features.0.0.weight
	 features.0.1.weight
	 features.0.1.bias
	 features.1.0.block.0.0.weight
	 features.1.0.block.0.1.weight
	 features.1.0.block.0.1.bias
	 features.1.0.block.1.fc1.weight
	 features.1.0.block.1.fc1.bias
	 features.1.0.block.1.fc2.weight
	 features.1.0.block.1.fc2.bias
	 features.1.0.block.2.0.weight
	 features.1.0.block.2.1.weight
	 features.1.0.block.2.1.bias
	 features.1.1.block.0.0.weight
	 features.1.1.block.0.1.weight
	 features.1.1.block.0.1.bias
	 features.1.1.block.1.fc1.weight
	 features.1.1.block.1.fc1.bias
	 features.1.1.block.1.fc2.weight
	 features.1.1.block.1.fc2.bias
	 features.1.1.block.2.0.weight
	 features.1.1.block.2.1.weight
	 features.1.1.block.2.1.bias
	 features.2.0.block.0.0.weight
	 features.2.0.block.0.1.weight
	 features.2.0.block.0.1.bias
	 features.2.0.block.1.0.weight
	 features.2.0.block.1.1.weight
	 features.2.0.block.1.1.bias
	 features.2.0.block.2.fc1.weight
	 features.2.0.block.2.fc1.bias
	 features.2.0.blo

In [11]:
# 17 min 1 epoch - 128 batch size - inception
# Efficientnet b0 - batch 96 - epoch 50 - num_workers 2 - flip, auto cont, sharp - 

In [12]:
# Observe that all parameters are being optimized
optimizer = optim.Adam(params_to_update, lr=lr_start)

# Learning rate scheduler.
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=lr_end, last_epoch=-1)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_pyt, prof, val_history, train_history = train_model(device, model_pyt, data_loaders, 
                                                            optimizer, scheduler,
                                                            criterion, 
                                                            num_epochs=num_epochs,
                                                            num_classes=num_classes,
                                                            is_inception=(model_name=="inceptionv3"),
                                                            profiler=False)

Epoch 1/100
----------
Epoch [1/100], Step [100/188], Loss: 0.2117, Accuracy: 29.00%
train Loss: 0.4935 Acc: 0.8131
val Loss: 0.5247 Acc: 0.8205

Epoch 2/100
----------
Epoch [2/100], Step [100/188], Loss: 0.2720, Accuracy: 30.00%
train Loss: 0.2344 Acc: 0.9129
val Loss: 0.3030 Acc: 0.8990

Epoch 3/100
----------
Epoch [3/100], Step [100/188], Loss: 0.0801, Accuracy: 31.00%
train Loss: 0.2148 Acc: 0.9189
val Loss: 0.3561 Acc: 0.8664

Epoch 4/100
----------
Epoch [4/100], Step [100/188], Loss: 0.0567, Accuracy: 32.00%
train Loss: 0.1565 Acc: 0.9415
val Loss: 0.2209 Acc: 0.9340

Epoch 5/100
----------
Epoch [5/100], Step [100/188], Loss: 0.2610, Accuracy: 30.00%
train Loss: 0.1773 Acc: 0.9382
val Loss: 0.2242 Acc: 0.9293

Epoch 6/100
----------
Epoch [6/100], Step [100/188], Loss: 0.0598, Accuracy: 31.00%
train Loss: 0.1352 Acc: 0.9542
val Loss: 0.2388 Acc: 0.9169

Epoch 7/100
----------
Epoch [7/100], Step [100/188], Loss: 0.1128, Accuracy: 31.00%
train Loss: 0.1193 Acc: 0.9585
val Loss

In [13]:
dataset_name = 'covid_pneumonia'

In [14]:
save_model(model_pyt, '../../models/', 
           f'Transfer_{model_name}_{dataset_name}_{num_epochs}_{batch_size}_{lr_start}_{lr_end}_model_weights.pth')

True

In [15]:
eval_metrics = eval_model(device=device, model=model_pyt, test_loader=data_loaders['test'], is_inception=(model_name=="inceptionv3"))

In [16]:
for i, v in eval_metrics.items():
    print(f"{i}: {v}")

acc: 0.9797979797979798
f1: 0.9783785379966549
cm: [[533   9   0]
 [  6 530   7]
 [  2   2 198]]
outputs: [0 0 0 ... 2 2 2]
targets: [0 0 0 ... 2 2 2]


In [17]:
with open(f'../../models/Transfer_val_history_{model_name}_{dataset_name}_{num_epochs}_{batch_size}_{lr_start}_{lr_end}_.pickle', 'wb') as handle:
    pickle.dump(val_history, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'../../models/Transfer_train_history_{model_name}_{dataset_name}_{num_epochs}_{batch_size}_{lr_start}_{lr_end}_.pickle', 'wb') as handle:
    pickle.dump(train_history, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'../../models/Transfer_eval_metrics_{model_name}_{dataset_name}_{num_epochs}_{batch_size}_{lr_start}_{lr_end}_.pickle', 'wb') as handle:
    pickle.dump(eval_metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('filename.pickle', 'rb') as handle:
#    b = pickle.load(handle)

In [18]:
#print(prof.key_averages(group_by_stack_n=5).table(sort_by='self_cpu_time_total', row_limit=50))

In [19]:
# Plot the training curves of validation accuracy vs. number
#  of training epochs for the transfer learning method and
#  the model trained from scratch
# vhist = []
# vhist = [h.cpu().numpy() for h in val_acc_history]
# thist = []
# thist = [h.cpu().numpy() for h in train_acc_history]

# plt.title("Accuracy vs. Number of Training Epochs")
# plt.xlabel("Training Epochs")
# plt.ylabel("Accuracy")
# #plt.plot(range(1,num_epochs+1),ohist,label="Pretrained")
# plt.plot(range(1,num_epochs+1),vhist,label="Validation")
# plt.plot(range(1,num_epochs+1),thist,label="Training")
# plt.ylim((0,1.))
# plt.xticks(np.arange(1, num_epochs+1, 1.0))
# plt.legend()
# plt.show()

In [20]:
# Plot the training curves of validation accuracy vs. number
#  of training epochs for the transfer learning method and
#  the model trained from scratch
# vhist = []
# vhist = [h for h in val_loss_history]
# thist = []
# thist = [h for h in train_loss_history]

# plt.title("Loss vs. Number of Training Epochs")
# plt.xlabel("Training Epochs")
# plt.ylabel("Loss")
# #plt.plot(range(1,num_epochs+1),ohist,label="Pretrained")
# plt.plot(range(1,num_epochs+1),vhist,label="Validation")
# plt.plot(range(1,num_epochs+1),thist,label="Training")
# plt.ylim((0,1.))
# plt.xticks(np.arange(1, num_epochs+1, 1.0))
# plt.legend()
# plt.show()